# Human Activity Recognition based on Smartphones sensors.
---
<b>MADS-MMS Portfolio-Exam Part 2<br>
Janosch Höfer, 938969</b>

## Table of contents

- [Introduction](#intro) <br>
- [1. Data preparation](#data-prep) <br>
- [2. Parameters](#parameters) <br>
- [3. Model setup and Crossvalidation](#model-setup) <br>
   - [3.1. Bla?](#NN) <br>
- [4. Model Evaluation](#model-eval) <br>
- [5. Results](#results)<br>
- [References](#ref)<br>

## Introduction

bla<br>
Using [[1]](http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions)

In [ ]:
# Standard libraries
import os

import numpy as np
import pandas as pd
import re

# Own classes and functions
from helper_functions.data_manipulation import (
    create_activity_window,
    open_raw_data,
    setup_raw_data,
)

---
<a id='data-prep'></a>

## 1. Data preparation

bla

In [ ]:
path_to_data = "data"  # dir where the zip has been extracted
data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00341/HAPT%20Data%20Set.zip"
experiment_types = ["acc", "gyro"]

In [ ]:
# Check for data
if os.path.isdir(path_to_data):
    dir_count = [_dir for _dir in os.listdir(path_to_data) if _dir in ["RawData", "Test", "Train"]]
    if len(dir_count) != 3:
        print("Data folder found, but it does not contain the right data. Downloading...")
        setup_raw_data(data_url, path_to_data)
    else:
        print("Found the correct data.")
else:
    print("Data not found. Downloading...")
    setup_raw_data(data_url, path_to_data)

In [ ]:
labels = pd.read_csv(
    os.path.join(path_to_data, "RawData", "labels.txt"),
    header=None,
    sep=" ",
    names=["experiment_id", "user_id", "activity_id", "start", "end"],
)

In [ ]:
labels.head()

Example data

In [ ]:
# Example code used by underlying functions
experiment_data = {
    exp_type: open_raw_data(path_to_data, exp_type, 1, 1) for exp_type in experiment_types
}
exp_types = list(experiment_data.keys())
first_type = exp_types.pop(0)
_df = pd.DataFrame(experiment_data[first_type]).add_prefix(f"{first_type}_")
for sub_type in exp_types:
    _df = pd.merge(
        _df,
        pd.DataFrame(experiment_data[sub_type]).add_prefix(f"{sub_type}_"),
        left_index=True,
        right_index=True,
    )
_df.head()

Example data single row

In [ ]:
# Example code used by underlying functions
agg_funcs = [np.mean, np.min, np.max]
df_stacked = _df.iloc[250:500, :].agg(agg_funcs).stack().swaplevel()
df_stacked.index = df_stacked.index.map("{0[1]}_{0[0]}".format)
df_stacked.to_frame().T

In [ ]:
agg_functions = [np.mean, np.min, np.max]

exp_results = list()
for (exp_id, user_id), activity_df in labels.groupby(["experiment_id", "user_id"]):
    experiment_data = {
        exp_type: open_raw_data(path_to_data, exp_type, exp_id, user_id)
        for exp_type in experiment_types
    }
    for act_idx in range(len(activity_df)):
        _df = create_activity_window(
            experiment_data,
            agg_functions,
            activity_df.iloc[act_idx]["start"],
            activity_df.iloc[act_idx]["end"],
        )
        _df["exp_id"] = exp_id
        _df["user_id"] = user_id
        _df["activity_id"] = activity_df.iloc[act_idx]["activity_id"]
        exp_results.append(_df)
df = pd.concat(exp_results)

In [ ]:
df

Loading the prepared data.

In [ ]:
feature_dir = os.path.join(path_to_data, "features.txt")
with open(feature_dir) as file:
    features = [row.rstrip("\n").rstrip() for row in file]

In [ ]:
len(features), len(set(features))

In [ ]:
replace_dict = {"ropy": [1, ""], "Kurtosis": [1, ""], "Skewness": [1, ""]}
new_features = list()
for row in features:
    reg_idx = re.findall("[^-]*[^-]", row)[1]
    if replace_dict.get(reg_idx):
        if row == replace_dict[reg_idx][1]:
            replace_dict[reg_idx][0] += 1
            row = row.replace("-1", f"-{replace_dict[reg_idx][0]}")
        else:
            replace_dict[reg_idx][0] = 1
            replace_dict[reg_idx][1] = row
    new_features.append(row)

In [ ]:
len(features), len(set(new_features))

In [ ]:
data_path = os.path.join(path_to_data, "Train", "X_train.txt")
df_full = pd.read_csv(data_path, sep=" ", header=None, names=new_features)

In [ ]:
df_full

---
<a id='parameters'></a>

## 2. Parameters

bla

---
<a id='split'></a>

## 3. Model setup and Crossvalidation
<a id='NN'></a>
### 3.1 Bla?

bla

---
<a id='model-eval'></a>

## 4. Model Evaluation

bla

---
<a id='results'></a>

## 5. Results

bla

---
<a id='ref'></a>

## References

<p> [1] http://archive.ics.uci.edu/ml/datasets/Smartphone-Based+Recognition+of+Human+Activities+and+Postural+Transitions